## Interacting with csv files

In [1]:
import os 
import pandas as pd

from IPython.display import Markdown, HTML, display
from langchain.schema import HumanMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

from agentops.partners.langchain_callback_handler import (
    LangchainCallbackHandler as AgentOpsLangchainCallbackHandler,
)



In [2]:
df = pd.read_csv("./data/all-states-history.csv").fillna(value = 0)

### Prepare Langchain pandas agent

In [3]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

AGENTOPS_API_KEY = os.getenv("AGENTOPS_API_KEY")

agentops_handler = AgentOpsLangchainCallbackHandler(
    api_key=AGENTOPS_API_KEY, default_tags=["Langchain Example"]
)

llm = ChatOpenAI(
    model="gpt-4o-mini", 
    temperature=0,
    callbacks=[agentops_handler]
)

# You must pass in a callback handler to record your agent
llm.callbacks = [agentops_handler]

agent = create_pandas_dataframe_agent(
    llm,
    df=df,
    agent_type="tool-calling",
    allow_dangerous_code=True,
    verbose=True
)

agent.invoke("how many rows are there?")



🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=7fa66339-5738-4a37-acfe-9e618ae90a47




> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'len(df)'}`


20780There are 20,780 rows in the dataframe.

> Finished chain.


{'input': 'how many rows are there?',
 'output': 'There are 20,780 rows in the dataframe.'}

### Configure the question and ask it

In [4]:
CSV_PROMPT_PREFIX = """
First set the pandas display options to show all the columns,
get the column names, then answer the question.
"""

CSV_PROMPT_SUFFIX = """
- **ALWAYS** before giving the Final Answer, try another method.
Then reflect on the answers of the two methods you did and ask yourself
if it answers correctly the original question.
If you are not sure, try another method.
- If the methods tried do not give the same result,reflect and
try again until you have two methods that have the same result.
- If you still cannot arrive to a consistent result, say that
you are not sure of the answer.
- If you are sure of the correct answer, create a beautiful
and thorough response using Markdown.
- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,
ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.
- **ALWAYS**, as part of your "Final Answer", explain how you got
to the answer on a section that starts with: "\n\nExplanation:\n".
In the explanation, mention the column names that you used to get
to the final answer.
"""

QUESTION = """How may patients were hospitalized during July 2020
in Florida, and nationwide as the total of all states?
Use the hospitalizedIncrease column"""

agent.invoke(CSV_PROMPT_PREFIX + QUESTION + CSV_PROMPT_SUFFIX)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "import pandas as pd\n\n# Set display options to show all columns\npd.set_option('display.max_columns', None)\n\n# Get the column names\ncolumn_names = df.columns.tolist()\ncolumn_names"}`


['date', 'state', 'death', 'deathConfirmed', 'deathIncrease', 'deathProbable', 'hospitalized', 'hospitalizedCumulative', 'hospitalizedCurrently', 'hospitalizedIncrease', 'inIcuCumulative', 'inIcuCurrently', 'negative', 'negativeIncrease', 'negativeTestsAntibody', 'negativeTestsPeopleAntibody', 'negativeTestsViral', 'onVentilatorCumulative', 'onVentilatorCurrently', 'positive', 'positiveCasesViral', 'positiveIncrease', 'positiveScore', 'positiveTestsAntibody', 'positiveTestsAntigen', 'positiveTestsPeopleAntibody', 'positiveTestsPeopleAntigen', 'positiveTestsViral', 'recovered', 'totalTestEncountersViral', 'totalTestEncountersViralIncrease', 'totalTestResults', 'totalTestResultsIncrease', 'totalTestsAntibody', 'total

{'input': '\nFirst set the pandas display options to show all the columns,\nget the column names, then answer the question.\nHow may patients were hospitalized during July 2020\nin Florida, and nationwide as the total of all states?\nUse the hospitalizedIncrease column\n- **ALWAYS** before giving the Final Answer, try another method.\nThen reflect on the answers of the two methods you did and ask yourself\nif it answers correctly the original question.\nIf you are not sure, try another method.\n- If the methods tried do not give the same result,reflect and\ntry again until you have two methods that have the same result.\n- If you still cannot arrive to a consistent result, say that\nyou are not sure of the answer.\n- If you are sure of the correct answer, create a beautiful\nand thorough response using Markdown.\n- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,\nONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.\n- **ALWAYS**, as part of your "Final Answer", explain how you 